In [1]:
import os
import numpy as np

import tensorflow as tf

In [18]:
# create a list to all tensorboard log file
# logs_directory = "../../tmp/tensorboard"
logs_directory = "../standalone/tensorboard/cross_validation/"

all_log_dir = os.listdir(logs_directory)

all_log_path = []
for log_dir in all_log_dir:
    log_dir_path = os.path.join(logs_directory, log_dir)
    all_log_path.append(os.path.join(log_dir_path, os.listdir(log_dir_path)[0]))

In [19]:
tags = ["train/acc_1", "val/acc_1"]

def get_log_data(path):
    logs = dict(zip(tags, [[] for _ in range(len(tags))]))

    for e in tf.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag in tags:
                    logs[value.tag].append(value.simple_value)
    
    return logs

# Get all the logs

In [20]:
all_logs = dict(zip(all_log_path, [None for _ in range(len(all_log_path))]))

# for path in all_log_path:
for path in all_log_path:
    all_logs[path] = get_log_data(path)


# Calculate the stats for all the logs

In [26]:
all_logs_stats = dict(zip(all_log_path, [{} for _ in range(len(all_log_path))]))

for path in all_log_path:
    for tag in all_logs[path]:
        try: 
            all_logs_stats[path][tag] = {"max": None, "min": None}
            all_logs_stats[path][tag]["max"] = np.asarray(all_logs[path][tag]).max()
            all_logs_stats[path][tag]["min"] = np.asarray(all_logs[path][tag]).min()
        except ValueError as e:
            print("Error for tag %s in path %s" % (tag, path))
            print("continue...")
            pass

Error for tag train/acc_1 in path ../standalone/tensorboard/cross_validation/2019-12-21_02:42:03_run9_0.01lr_0.1e_5lcm_0.25ldm_160.0wl/events.out.tfevents.1576892523.lcances.20741.0
continue...
Error for tag val/acc_1 in path ../standalone/tensorboard/cross_validation/2019-12-21_02:42:03_run9_0.01lr_0.1e_5lcm_0.25ldm_160.0wl/events.out.tfevents.1576892523.lcances.20741.0
continue...


In [37]:
# search in the stat
def get_of(tag, key, search: str = None) -> list:
    output = []
    
    for path in all_logs_stats.keys():
        if search is not None:
            if search in path:
                output.append(all_logs_stats[path][tag][key])
        else:
            output.append((path, all_logs_stats[path][tag][key]))
    
    return output

In [72]:
target = "max"
response = get_of("val/acc_1", target)

# trim path from everythinfg irelevant
# response = [(response[i][0].split("/")[4].split("_")[2], response[i][1]) for i in range(len(response))]

# order by run
response = sorted(response, key=lambda x: x[0].split("/")[4].split("_")[2])

values = np.array([v[1] for v in response])
path = [p[0] for p in response]

values = np.array([i for i in values if i is not None])
print("Overall statistics --------")
print("mean: ", values.mean())
print("std:  ", values.std())
print("min:  ", values.min())
print("max:  ", values.max())
print("")
print("%s values --------" % target)
for p, v in response:
    print(p.split("/")[4].split("_")[2], v)

Overall statistics --------
mean:  0.5571969217724271
std:   0.04676416538156036
min:   0.5131264925003052
max:   0.6397058963775635

max values --------
run1 0.59617680311203
run10 0.517754852771759
run2 0.6397058963775635
run3 0.6203473806381226
run4 0.5131264925003052
run5 0.5249088406562805
run6 0.561965823173523
run7 0.5272727012634277
run8 0.5135135054588318
run9 None
